In [1]:
%matplotlib widget
import numpy as np
import os
import h5py
import subprocess

print(os.getpid())
%cd ../
import pylib.mix as mix
%cd "./tools/QSP-angles/"

30078
/media/work/docs/codes/QuCF/scripts-py
/media/work/docs/codes/QuCF/scripts-py/tools/QSP-angles


Compute angles (phase factors) for the QSP circuit in the formulation of [Low&Chung_2019](https://quantum-journal.org/papers/q-2019-07-12-163/) using the [Haah algorithm](https://quantum-journal.org/papers/q-2019-10-07-190/).

In [2]:
mix.reload_module(mix)
# -----------------------------------------------------------
def find_exp(number) -> int:
    base10 = np.log10(abs(number))

    temp = np.floor(base10)
    flag_neg = False
    if temp < 0:
        flag_neg = True

    return abs(temp), flag_neg
# -----------------------------------------------------------
def txt_to_hdf5(old_filename, new_filename):
    function_type_res = "QSP-ham"
    rescaling_factor_res = 1.
    parity_res = -1

    # --- read the .angles (text) file ---
    print("read angles from:\n " + old_filename)
    with open(old_filename) as f:
        param_res = float(f.readline()[:-1]) 
        eps_res   = float(f.readline()[:-1]) 
        Na        = int(f.readline()[:-1]) 
        angles = np.zeros(Na)
        for ii in range(Na):
            angles[ii] = float(f.readline())
            
    print()
    print("function type: \t\t{:s}".format(function_type_res))    
    print("rescaling factor: \t{:0.3f}".format(rescaling_factor_res))
    print("parity: \t\t{:d}".format(parity_res))
    print("parameter: \t\t{:0.3e}".format(param_res))
    print("error: \t\t\t{:0.3e}".format(eps_res))
    print("N-angles: \t\t{:d}".format(Na))
    print()       

    # --- write the .hdf5 file ---
    print("write angles to:\n " + new_filename)
    with h5py.File(new_filename, "w") as f:
        grp = f.create_group("basic")
        
        grp.create_dataset('polynomial_type',  data=function_type_res)
        grp.create_dataset('rescaling_factor', data=float(rescaling_factor_res))
        grp.create_dataset('parity',           data=parity_res)
        grp.create_dataset('eps',              data=eps_res)
        grp.create_dataset('par',              data=param_res)

        grp = f.create_group("angles")
        grp.create_dataset('QSP-ham',  data = angles)
    return

In [3]:
# ----------------------------------------------------------
# --- Specify QSP parameters ---
# ----------------------------------------------------------
t_moments = [0.1]

Nt = len(t_moments)
eps_QSP_array = [1e-12] * Nt

case_format = 1

# --- Form the name of the output name ---
file_names = [None] * Nt
print("--- Chosen QSP parameters ---")
if case_format == 1:
    for it in range(Nt):
        eps_QSP = eps_QSP_array[it]
        t1 = t_moments[it]
        exp_t1, flag_neg = find_exp(t1)

        str_exp = "p"
        if flag_neg:
            str_exp = "m"

        if flag_neg:
            temp_int = int(t1 * 10**(exp_t1+3))
        else:
            temp_int = int(t1 / 10**(exp_t1-3))

        file_names[it] = "QSP_t{:d}{:s}{:d}_eps{:d}".format(
            temp_int, 
            str_exp,
            int(exp_t1),
            int(abs(np.round(np.log10(eps_QSP))))
        )

        print("t, eps: {:0.12e}, {:0.3e}".format(t1, eps_QSP))
        print("file name: {:s}".format(file_names[it]))
        print()

--- Chosen QSP parameters ---
t, eps: 1.000000000000e-01, 1.000e-12
file name: QSP_t1000m1_eps12



In [4]:
# ------------------------------------------------
# --- Compute QSP angles ---
# ------------------------------------------------
line_code = "../compute-QSP-angles/test-qsp/bin/Debug/netcoreapp3.1/qsp"
for it in range(Nt):
    eps_QSP = eps_QSP_array[it]
    t1 = t_moments[it]

    line_cmd = "{:s} {:0.12e} {:0.3e} {:s} {:s}".format(
        line_code, t1, eps_QSP, "./", file_names[it]
    )
    subprocess.run(line_cmd, shell = True, executable="/bin/bash")

    # from the .txt file to .hdf5 file:
    txt_to_hdf5(file_names[it], file_names[it] + ".hdf5")

    # remove the .txt file:
    line_cmd = "rm {:s}".format(file_names[it])
    subprocess.run(line_cmd, shell = True, executable="/bin/bash")

Start calculating...
Chosen tau = 0.1
Chosen eps = 1E-12
polyA = { Denominator = 140737488355328
  Numerator =
             3053z^-6 + 36632066z^-4 + 175775304698z^-2 + 140385864475550z^0 + 175775304698z^2 + 36632066z^4 + 3053z^6
  Parity = 0 }
polyB = { Denominator = 140737488355328
  Numerator =
             21z^-7 + 366351z^-5 + 2930198946z^-3 + 7028081989012z^-1 + -7028081989012z^1 + -2930198946z^3 + -366351z^5 + -21z^7
  Parity = 1 }
yes, every exponent is even
Initially Found 14 roots out of 14
3 to refine, log2(err)=-24
0 to refine, log2(err)=-64
0 to refine, log2(err)=-72
alphaSqrt/2 = (+.708248421515318721241128586154541e-6,+.810321637012223356203719989264577e-38)

Resulting angles: 
3.141592442909515   3.1415618140100188   3.137886441904623   2.774435028888961   -0.2267245912408289   -2.880165336812886   -0.1461951995155765   2.74606058289201   0.012090445594701047   -2.641350299977356   0.3598093201524341   -0.0036420928632280897   3.327924216735339E-05   2.6503426709772785E